# DB2-Salesforce connector: URL issues

In [46]:
# API settings
api_url = '/services/data/v43.0/sobjects'
object_id = 'nanoHUB_issue__c'
external_id = 'Issue_ID__c'

# login parameters to be handled by Papermill
sql_login_params = {"username": "wang2506_ro", "password": "fnVnwcCS7iT45EsA"}
sf_login_params = {
    "grant_type": "password",
    "client_id": "3MVG95jctIhbyCppj0SNJ75IsZ1y8UPGZtSNF4j8FNVXz.De8Lu4jHm3rjRosAtsHy6qjHx3i4S_QbQzvBePG",
    "client_secret": "D1623C6D3607D4FC8004B92C761DFB6C1F70CCD129C5501E357028DFA00F5764",
    "username":"wang2506@purdue.edu",
    "password":"mathisfun992728tXYwCb5yMm6XtohbTRw8ZdBbi",    
} 
    #"username": "wang159-4j1v@force.com",
    #"password": "napoleon0eZ3PQpQqE3C3z4wWqEqKGhQ8",

In [48]:
import sys
sys.path.append('/home/users/wang2506/nanohub_salesforce_integ/salesforce')
    
import pandas as pd
import datetime

## Obtain tool information from DB2 

In [49]:
# Basic tool information
sql_query = "select * from issue_invalid_urls"

# display
print(sql_query)

select * from issue_invalid_urls


In [50]:
# connect with DB2
import sqlalchemy as sql

engine = sql.create_engine('mysql+pymysql://%s:%s@127.0.0.1/wang159_myrmekes' \
                                               %(sql_login_params['username'], sql_login_params['password']))
df = pd.read_sql_query(sql_query, engine)

In [51]:
# display
df.head(2)

,index,resource_ID,href,href_text,status
0,0,90,http://epics.ecn.purdue.edu/lspm,Lego Scanning Probe Microscope,404
1,1,90,http://www.inac.purdue.edu/education/lspm/exhi...,Lego SPM website,Connection error.


In [66]:
df_export = pd.DataFrame()

# Make sure NaN and NaT values are taken care of here

# construct a consistent ID for this issue
df_export[external_id]      =  'Resource_URL::::'+df.resource_ID+'::::'+df.href+'::::'+df.href_text

df_export['Resource ID'] = df['resource_ID']
df_export['Resource URL'] = 'https://nanohub.org/resources/'+df['resource_ID']
df_export['Reason']           = 'Invalid URL'

df_export['Invalid URL label'] = df['href_text']
df_export['Invalid URL'] = df['href']

# display
df_export.to_excel('./invalid_URL_export.xlsx', engine='xlsxwriter')

## Match data with Salesforce format

In [52]:
df_sf = pd.DataFrame()

# Make sure NaN and NaT values are taken care of here

# construct a consistent ID for this issue
df_sf[external_id]      =  'Resource_URL::::'+df.resource_ID+'::::'+df.href+'::::'+df.href_text

df_sf['nanoHUB_resource_ID__c'] = df['resource_ID']
df_sf['Origin_URL__c'] = 'https://nanohub.org/resources/'+df['resource_ID']
df_sf['Type__c']           = 'Invalid URL'

df_sf['Issue_message__c'] = df.apply(lambda x: \
         'Invalid URL found on page.<br><br>\
         Link label: <p style="color:Tomato;">%s</p><br>\
         Link: <p style="color:Tomato;">%s</p>'%(x.href_text, x.href), axis=1)

sf_original_fields = df_sf.columns

# display
df_sf.head(2).T

,0,1
Issue_ID__c,Resource_URL::::90::::http://epics.ecn.purdue....,Resource_URL::::90::::http://www.inac.purdue.e...
nanoHUB_resource_ID__c,90,90
Origin_URL__c,https://nanohub.org/resources/90,https://nanohub.org/resources/90
Type__c,Invalid URL,Invalid URL
Issue_message__c,Invalid URL found on page.<br><br> Lin...,Invalid URL found on page.<br><br> Lin...


## To Salesforce Sales Cloud CRM

In [57]:
from DB2SalesforceAPI import DB2SalesforceAPI

# create DB2 to Salesforce API object
db_s = DB2SalesforceAPI(sf_login_params)

# specify Salesforce object ID and external ID
db_s.object_id = object_id
db_s.external_id = external_id

Obtained Salesforce access token ...... True


In [58]:
# send data to Salesforce
db_s.send_data(df_sf)

[Success] Bulk job creation successful. Job ID = 7505w00000M0insAAB
[Success] CSV upload successful. Job ID = 7505w00000M0insAAB
[Success] Closing job successful. Job ID = 7505w00000M0insAAB


In [59]:
# check status
db_s.check_bulk_status()

{'id': '7505w00000M0insAAB',
 'operation': 'upsert',
 'object': 'nanoHUB_issue__c',
 'createdById': '0055w00000ArpYvAAJ',
 'createdDate': '2020-05-11T05:19:25.000+0000',
 'systemModstamp': '2020-05-11T05:19:30.000+0000',
 'state': 'JobComplete',
 'externalIdFieldName': 'Issue_ID__c',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 1403,
 'numberRecordsFailed': 26,
 'retries': 0,
 'totalProcessingTime': 3373,
 'apiActiveProcessingTime': 2665,
 'apexProcessingTime': 0}

In [60]:
from pprint import pprint
pprint(db_s.check_bulk_failed_results())

('"sf__Id","sf__Error",Issue_ID__c,Issue_message__c,Origin_URL__c,Type__c,nanoHUB_resource_ID__c\n'
 '"","DUPLICATE_VALUE:Duplicate external id specified: '
 'resource_url::::124::::http://www.elsevier.com/wps/product/cws_home/680682::::academic '
 'press, 2003, isbn: 0-12-507060-8:Issue_ID__c '
 '--","Resource_URL::::124::::http://www.elsevier.com/wps/product/cws_home/680682::::Academic '
 'Press, 2003, ISBN: 0-12-507060-8","Invalid URL found on '
 'page.<br><br>         Link label: <p style=""color:Tomato;"">Academic Press, '
 '2003, ISBN: 0-12-507060-8</p><br>         Link: <p '
 'style=""color:Tomato;"">http://www.elsevier.com/wps/product/cws_home/680682</p>","https://nanohub.org/resources/124","Invalid '
 'URL","124.0"\n'
 '"","DUPLICATE_VALUE:Duplicate external id specified: '
 'resource_url::::124::::http://www.elsevier.com/wps/product/cws_home/680682::::academic '
 'press, 2003, isbn: 0-12-507060-8:Issue_ID__c '
 '--","Resource_URL::::124::::http://www.elsevier.com/wps/product/